In [188]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [192]:
TICKER = 'SPY'
WINDOW = 3
FAST = 10
SLOW = 27

def get_data():
    df = yf.download(TICKER, period='1y')#, auto_adjust=False)
    df.columns = df.columns.get_level_values(0)
    
    return df

def add_moving_averages(df, fast, slow):
    df[f'{FAST}_ma'] = df['Close'].rolling(fast).mean()
    df[f'{SLOW}_ma'] = df['Close'].rolling(slow).mean()
    return df.dropna()

df = get_data()
df = add_moving_averages(df, FAST, SLOW)

df


/tmp/ipykernel_48851/2433912205.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(TICKER, period='1y')#, auto_adjust=False)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,10_ma,27_ma
Date,,,,,,,
2024-08-12,526.709778,529.139477,524.418311,527.638216,42542100,525.832684,538.829556
2024-08-13,535.371826,535.608924,529.682736,529.929661,52333100,525.836633,538.368627
2024-08-14,537.060852,538.255988,533.475503,536.171900,42446900,525.139319,537.946843
2024-08-15,546.266113,546.552524,542.127656,542.740024,60846800,526.132941,537.846241
2024-08-16,547.490906,548.192193,544.478439,544.636444,44430700,528.247601,537.589805
...,...,...,...,...,...,...,...
2025-06-27,614.909973,616.390015,610.830017,612.880005,86258400,602.270172,595.333607
2025-06-30,617.849976,619.219971,615.039978,617.380005,92502500,604.531140,596.324253
2025-07-01,617.650024,618.830017,615.520020,616.359985,70030100,606.205786,597.676403


In [190]:
def add_strategy(df):
    df.columns = df.columns.get_level_values(0)
    df['Strategy'] = np.where(df['Close'] > df['MA'], 1, -1)
    df['Strategy'] = df['Strategy'].shift(1)
    return df

def test_strategy(df):
    df['Asset_Culmulative']  = np.cumprod(1+df['Close'].pct_change()) -1
    df['Strategy_Culmulative'] = np.cumprod(1+df['Close'].pct_change() * df['Strategy']) -1
    
    plt.plot(df['Asset_Culmulative'])
    plt.plot(df['Strategy_Culmulative'])
    plt.title(f'{TICKER} Close vs Moving Average')
    plt.legend([f'{TICKER} Close Price', f'{WINDOW} Moving Average'])
    
    return df

def main():
    df = get_data()
    df = add_strategy(df)
    df = test_strategy(df)
    return df

main()

/tmp/ipykernel_48851/504764323.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(TICKER)#, auto_adjust=False)
[*********************100%***********************]  1 of 1 completed


KeyError: 'MA'